In [158]:
import numpy as np
from pandas import Series, DataFrame

In [159]:
def dis(a,b):
    d=np.sum((a[None,:] - b[:, None])**2, -1)**0.5
    return d

In [160]:
def cost(a,b):
    return sum(dis(a,b).min(axis=0))

In [187]:
def modelcost(center,cluster):
    tot=[sum(dis(center[i],cluster[i])) for i in range(len(center))]
    return sum(tot)

In [161]:
def kmeans(data, ncenter, center, maxiter=1000, maxtol=10e-5):
    centroids=[center]
    i = 0
    diff=100
    while (diff>maxtol and i<maxiter):
        # assign data points to clusters
        
        index=np.argmin(dis(data,centroids[i]),axis=0)
        
        clusters=[data[index==i] for i in range(ncenter)]

        # recalculate centroids
        centroids.append(np.concatenate([np.mean(cluster,axis=0) for cluster in clusters]).reshape(ncenter,-1))
        diff=sum(np.sum((centroids[i+1]-centroids[i])**2,-1)**0.5)
        i+= 1

    return centroids, clusters,i

In [162]:
def kpp(x,k,weight=1):
    index=[]
    index.append(np.random.choice(x.shape[0],1)[0])
    while(len(index)<k):
        prob=dis(x,x[index,]).min(axis=0)*weight/sum(dis(x,x[index,]).min(axis=0)*weight)  
        index.append(np.random.choice(x.shape[0],1,p=prob)[0])
    return x[index,]

In [163]:
def kII(x,k,l):
    index=set()
    index.add(np.random.choice(x.shape[0],1)[0])
    cost=sum(dis(x,x[list(index),]).min(axis=0))
    i=1
    while (i<np.log(cost)):
        prob=dis(x,x[list(index),]).min(axis=0)/sum(dis(x,x[list(index),]).min(axis=0)) 
        cp=np.random.choice(x.shape[0],l,p=prob,replace=True)
        index=index.union(set(cp))
        i=i+1
    center0=x[list(index),]
    w=np.sum(np.argsort(dis(center0,center0),axis=1)==1,axis=0)
    center1=kpp(center0,k,w)
    return center1

In [164]:
#simulation
def simudata(ncenter,nsize,dim):
    mean=[0]*dim
    cov=np.diag(np.random.choice([1,10,100],dim,replace=True))
    center=np.random.multivariate_normal(mean, cov, ncenter)
    s=np.random.multivariate_normal(center[0,], np.eye(dim),nsize)
    sample=np.concatenate((s, 0*np.ones(nsize)[:,None]), axis=1)
    for i in range(ncenter-1):
        s=np.random.multivariate_normal(center[i+1,], np.eye(dim),nsize)
        s=np.concatenate((s, (i+1)*np.ones(nsize)[:,None]), axis=1)
        sample=np.concatenate((sample,s),axis=0)
    return center, sample

In [165]:
center, sample=simudata(20,1000,15)
sample
data=sample[:,0:15]

In [166]:
center0=kpp(sample[:,0:15],20)

In [167]:
center1=kII(sample[:,0:15],20,10)

In [168]:
center,cluster, iter=kmeans(data, ncenter=20, center=center1, maxiter=1000, maxtol=10e-5)

In [189]:
modelcost(center[-1],cluster)

array([ 122006.90388946])

In [190]:
center,cluster, iter=kmeans(data, ncenter=20, center=center0, maxiter=1000, maxtol=10e-5)

/Users/aprilwang/anaconda/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
